In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import gzip
import seaborn as sns
import bz2
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import gensim.models.word2vec

In [3]:
# Parameters
walk_size = 50
n_walks = 1000000
tuple_file = '/home/riri/Datasets/dbpedia/page_links_en_uris_nl.ttl.bz2'

In [4]:
url_len = len('http://dbpedia.org/resource/ ')

links = defaultdict(set)
mapping = {}
k = 0

with bz2.BZ2File(tuple_file, 'r') as links_f:
    i = 0
    
    for l in tqdm(links_f, total=13495220):
        l = l.decode('utf-8')
        if l.startswith('#'):
            continue
        f, _, t, _ = l.split(' ')
        f = f[url_len:-1]
        t = t[url_len:-1]
        
        if f not in mapping:
            mapping[f] = k
            k += 1
        if t not in mapping:
            mapping[t] = k
            k += 1
        
        f = mapping[f]
        t = mapping[t]
        
        links[t].add(f)
        links[f].add(t)
        i += 1
# Turn all sets to lists.
for k in links.keys():
    links[k] = list(links[k])
    
inverse_mapping = {v:k for k,v in mapping.items()}

In [5]:
start_nodes = np.array(list(links.keys()))
def random_walk(j):
    node = np.random.choice(start_nodes)
    
    for _ in range(j):
        yield node
        if not links[node]:
            return
        node = np.random.choice(links[node])
        
def sent_gen():
    for l in tqdm(range(n_walks)):
        yield [inverse_mapping[n] for n in random_walk(walk_size)]

In [6]:
model = gensim.models.word2vec.Word2Vec(window=5, min_count=5, workers=4,)
model.build_vocab(sent_gen())

In [7]:
model.train(sent_gen())

49806419

In [44]:
model.vocab['Queen']

In [47]:
model.most_similar(positive=[u'Male', u'Queen'], negative=[u'Female'])

[(u'Hourglass', 0.9606831669807434),
 (u'Spinning_(textiles)', 0.957030177116394),
 (u'Category:Fairy_tales', 0.9566807746887207),
 (u'Cordwainer', 0.9565625190734863),
 (u'Combat_boot', 0.9560569524765015),
 (u'School_library', 0.9518604278564453),
 (u'Oude_Rode_Ogen', 0.9516125917434692),
 (u'Body_art', 0.9511663913726807),
 (u'Garland', 0.9507995247840881),
 (u'Magdeburg_hemispheres', 0.9503589868545532)]

In [35]:
model.most_similar([u'South', u'Canada'], [u'North'])

[(u'Carmangay', 0.8164716958999634),
 (u'Andorra_at_the_1976_Summer_Olympics', 0.8036561012268066),
 (u'Kitscoty', 0.7786150574684143),
 (u'Montreal', 0.7621344923973083),
 (u'Coderre,_Saskatchewan', 0.761287271976471),
 (u'Dewberry,_Alberta', 0.7602828145027161),
 (u'Falher', 0.7563831806182861),
 (u'New_Zealand', 0.755115270614624),
 (u'High_River', 0.7530913352966309),
 (u'Quebec', 0.7511137127876282)]